In [1]:
import pandas as pd
import gensim
import numpy as np
from sklearn.cluster import MiniBatchKMeans
from sklearn.metrics import silhouette_samples, silhouette_score

import os
import random
import re
import string

import nltk
import numpy as np
import pandas as pd

from gensim.models import Word2Vec

from nltk import word_tokenize
from nltk.corpus import stopwords

from sklearn.cluster import MiniBatchKMeans
from sklearn.metrics import silhouette_samples, silhouette_score

data = pd.read_csv('E:\DataSet.csv')
#data.head()
#data.info()
#data

In [2]:
data_set = data.Content.apply(gensim.utils.simple_preprocess)
#data_set

In [3]:
from gensim.models import Word2Vec
model = Word2Vec(sentences=data_set, vector_size=100, workers=1, seed=25)

In [23]:
model.wv.most_similar("arms")

[('offense', 0.9432849287986755),
 ('any', 0.9413777589797974),
 ('be', 0.9393042325973511),
 ('or', 0.9385942220687866),
 ('section', 0.9382218718528748),
 ('the', 0.9381684064865112),
 ('person', 0.938145101070404),
 ('to', 0.9380295872688293),
 ('of', 0.9373148679733276),
 ('with', 0.9367450475692749)]

In [24]:
model.wv.similarity(w1="arms", w2="child")

0.90473187

In [4]:
def vectorize(list_of_docs, model):
    """Generate vectors for list of documents using a Word Embedding

    Args:
        list_of_docs: List of documents
        model: Gensim's Word Embedding

    Returns:
        List of document vectors
    """
    features = []

    for tokens in list_of_docs:
        zero_vector = np.zeros(model.vector_size)
        vectors = []
        for token in tokens:
            if token in model.wv:
                try:
                    vectors.append(model.wv[token])
                except KeyError:
                    continue
        if vectors:
            vectors = np.asarray(vectors)
            avg_vec = vectors.mean(axis=0)
            features.append(avg_vec)
        else:
            features.append(zero_vector)
    return features
    
vectorized_docs = vectorize(data_set, model=model)
len(vectorized_docs), len(vectorized_docs[0])

(29, 100)

In [5]:
def mbkmeans_clusters(
	X, 
    k, 
    mb, 
    print_silhouette_values, 
):
    """Generate clusters and print Silhouette metrics using MBKmeans

    Args:
        X: Matrix of features.
        k: Number of clusters.
        mb: Size of mini-batches.
        print_silhouette_values: Print silhouette values per cluster.

    Returns:
        Trained clustering model and labels based on X.
    """
    km = MiniBatchKMeans(n_clusters=k, batch_size=mb).fit(X)
    print(f"For n_clusters = {k}")
    print(f"Silhouette coefficient: {silhouette_score(X, km.labels_):0.2f}")
    print(f"Inertia:{km.inertia_}")

    if print_silhouette_values:
        sample_silhouette_values = silhouette_samples(X, km.labels_)
        print(f"Silhouette values:")
        silhouette_values = []
        for i in range(k):
            cluster_silhouette_values = sample_silhouette_values[km.labels_ == i]
            silhouette_values.append(
                (
                    i,
                    cluster_silhouette_values.shape[0],
                    cluster_silhouette_values.mean(),
                    cluster_silhouette_values.min(),
                    cluster_silhouette_values.max(),
                )
            )
        silhouette_values = sorted(
            silhouette_values, key=lambda tup: tup[2], reverse=True
        )
        for s in silhouette_values:
            print(
                f"    Cluster {s[0]}: Size:{s[1]} | Avg:{s[2]:.2f} | Min:{s[3]:.2f} | Max: {s[4]:.2f}"
            )
    return km, km.labels_

In [6]:
clustering, cluster_labels = mbkmeans_clusters(
	X=vectorized_docs,
    k=3,
    mb=500,
    print_silhouette_values=True,
)
df_clusters = pd.DataFrame({
    "text": "docs",
    "tokens": [" ".join(text) for text in data_set],
    "cluster": cluster_labels
})

For n_clusters = 3
Silhouette coefficient: 0.50
Inertia:0.005138853754909152
Silhouette values:
    Cluster 2: Size:14 | Avg:0.54 | Min:-0.01 | Max: 0.67
    Cluster 1: Size:10 | Avg:0.46 | Min:0.02 | Max: 0.62
    Cluster 0: Size:5 | Avg:0.44 | Min:0.29 | Max: 0.57


C:\Users\DK23\anaconda3\lib\site-packages\sklearn\cluster\_kmeans.py:887: UserWarning: MiniBatchKMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can prevent it by setting batch_size >= 2048 or by setting the environment variable OMP_NUM_THREADS=2
  warnings.warn(


In [11]:
print("Most representative terms per cluster (based on centroids):")
for i in range(3):
    tokens_per_cluster = ""
    most_representative = model.wv.most_similar(positive=[clustering.cluster_centers_[i]], topn=10)
    for t in most_representative:
        tokens_per_cluster += f"{t[0]} "
    print(f"Cluster {i}: {tokens_per_cluster}")

Most representative terms per cluster (based on centroids):
Cluster 0: or of the to any with be section which for 
Cluster 1: or the to of any with be which section for 
Cluster 2: or the to of any with be which section for 


In [8]:
df_clusters

,text,tokens,cluster
0,docs,if any person kills or attempts to cause the d...,2
1,docs,if any person human trafficking prevention and...,1
2,docs,if person detains woman or child for ransom th...,1
3,docs,if man rapes woman or child he shall be punish...,1
4,docs,if woman commits suicide without the consent o...,2
5,docs,if person unlawfully touches or molestes woman...,1
6,docs,if woman husband or husband father mother guar...,1
7,docs,if person destroys or mutilates or mutilates c...,2
8,docs,notwithstanding anything different from any ot...,2
9,docs,news or information or name address or other i...,0


In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

exp_vals = [1,3,1]
exp_labels = ["Cluster 0","Cluster 1","Cluster"]

plt.pie(exp_vals,labels=exp_labels, shadow=True)
plt.axis("equal")
plt.show()